In [107]:
# general data managment
import numpy as np
import pandas as pd

# bokeh plotting
from bokeh.plotting import figure # actual plot
from bokeh.io import show, output_notebook, output_file # displaying plot
from bokeh.io import output_file,output_notebook, reset_output, save # for saving our graph
from bokeh.models import CategoricalColorMapper, HoverTool, ColumnDataSource, Panel # passive interactivity
from bokeh.models import MultiLine, Plot
from bokeh.models.widgets import CheckboxGroup, Slider, RangeSlider, Tabs # active interactivity
from bokeh.transform import linear_cmap

from bokeh.layouts import column, row, WidgetBox
from bokeh.palettes import Category20_16
from bokeh.application.handlers import FunctionHandler
from bokeh.application import Application

# tkinter
import tkinter as tk
from tkinter import ttk
from PIL import Image, ImageTk
from tkinter.filedialog import asksaveasfile
import os

# for opening html
import webbrowser

%run Spotify-ArtistReleases(a).ipynb

env: SPOTIPY_CLIENT_ID=d7975b1684ab45b7a28e97c355229077
env: SPOTIPY_CLIENT_SECRET=1b34039592a9481ca59d511da9d471a5
env: SPOTIPY_REDIRECT_URI=http://google.com/


In [115]:
# set output to notebook
# output_notebook()
# reset_output()

In [109]:
def search_API(dummy=1):
    '''
    If not authorized, first login
    Searches spotify API for artist and returns info in df format
    '''
    spotObj = authorize()
    artist_search(spotObj) # run artist search function

In [110]:
def add_artist(dummy=1):
    global all_data
    global current_data
    
    all_data = all_data.append(current_data)
    t = c_artists.cget("text")
    t += current_artist + '\n'
    c_artists.configure(text=t)

In [124]:
def generate_plots(dummy=1):
    '''
    Makes plot for included artists
    '''
    
    global all_data
    
    s = display
    s.configure(state=tk.NORMAL)
    s.insert(tk.END, 'generating plots...\n')
    s.configure(state=tk.DISABLED)
#     display.configure(text=s)
    
    def make_dataset(artist_list):
        by_artist = pd.DataFrame(columns=['artist','track_count','release','album_name','color'])
        
        for i, artist_name in enumerate(artist_list):
    
            # Subset to the artist
            subset = all_data[all_data['artist'] == artist_name]
            
            # create dataframe formatted for plotting
            arr_df = pd.DataFrame({'track_count': subset['track_count'], 'release': subset['release'],
                                  'album_name':subset['name']})
            
            # assign artist name for labels
            arr_df['artist'] = subset['artist']

            # color each artist differently
            arr_df['color'] = Category20_16[i]

            # Add to the overall dataframe
            by_artist = by_artist.append(arr_df)

        # Overall dataframe
        by_artist = by_artist.sort_values(['artist', 'release'])
        return by_artist
        
    def style(p):
        # Title 
        p.title.align = 'center'
        p.title.text_font_size = '20pt'
        p.title.text_font = 'serif'

        # Axis titles
        p.xaxis.axis_label_text_font_size = '14pt'
        p.xaxis.axis_label_text_font_style = 'bold'
        p.yaxis.axis_label_text_font_size = '14pt'
        p.yaxis.axis_label_text_font_style = 'bold'

        # Tick labels
        p.xaxis.major_label_text_font_size = '12pt'
        p.yaxis.major_label_text_font_size = '12pt'

        return p
    
    def make_plot(src):
        src['release'] = pd.to_datetime(src['release'], format='%Y-%m-%d')
        p = figure(plot_width = 700, plot_height = 700, 
                   title='Release Histories',
                   x_axis_label = 'Release Date', y_axis_label = 'Number of Tracks',
                   y_axis_type='linear',x_axis_type='datetime')
    
        artist_list = src['artist'].unique()
        # line glyphs to create a line graph of release history
        for i,artist in enumerate(artist_list):
            sub = src[src['artist'] == artist]
            src2 = ColumnDataSource(sub)
            p.line(x='release', y='track_count', line_color=sub['color'][0],
                   line_width=2, line_alpha = 0.7, legend_label = sub['artist'][0], 
                   hover_line_color = sub['color'][0], hover_line_alpha = 1.0,source=src2)
#         p.line(source=src, x='release', y='track_count', line_color = 'red',
#                      line_width=2, line_alpha = 0.7, legend_field = 'artist')#, 
#                      hover_line_color = 'color', 
#                      hover_line_alpha = 1.0)

        # Hover tool with vline mode
        hover = HoverTool(tooltips=[('Artist', '@artist'), 
                                    ('Album', '@album_name'),
                                    ('Release Date', '@release'),
                                    ('Number of Tracks', '@track_count')])
        p.add_tools(hover)
        p.legend.click_policy = 'hide'

        # Styling
        p = style(p)

        return p
        
    def update(attr, old, new):
        # Get the list of carriers for the graph
        artists_to_plot = [artist_selection.labels[i] for i in 
                            artist_selection.active]

        # Make a new dataset based on the selected artists
        new_src = make_dataset(artists_to_plot)
        
        # update the source used in the quad glpyhs
        src.data.update(new_src.data)
    
    
    # get list of available artists
    available_artists = list(all_data['artist'].unique())
    available_artists.sort()
    
    # define checbox instance from this list
    artist_selection = CheckboxGroup(labels=available_artists)
    artist_selection.on_change('active', update)
    
    controls = WidgetBox(artist_selection) # place checkboxes in widget
    
    src = make_dataset(available_artists)
    
    p = make_plot(src)

    # Create a row layout
    layout = row(controls, p)
    
    # Make a tab with the layout and add to doc
    tab = Panel(child=layout, title = 'Release Histories')
    tabs = Tabs(tabs=[tab])
    
    # set file name and save
    reset_output()
    output_file("artist_compare.html")
    save(tabs)
    
    # then open file
    new = 2 # open in a new tab, if possible
    webbrowser.open('artist_compare.html',new=new)

In [112]:
def save_data(dummy=1):
    global all_data
    
    s = display
    s.configure(state=tk.NORMAL)
    s.insert(tk.END, 'saving file...\n')
    s.configure(state=tk.DISABLED)
    
    files = [('All Files', '*.*'),  
             ('Comma-Seperated Values', '*.csv'), 
             ('Text Document', '*.txt')] 
    file = asksaveasfile(filetypes = files, defaultextension = files[1])
    try:
        all_data.to_csv(file,index=False)
    except:
        print("Couldn't write the file: ", file)
        raise 

In [113]:
class PlaceholderEntry(ttk.Entry):
    '''
    customizing placeholder text behavior
    '''
    def __init__(self, container, placeholder, *args, **kwargs):
        super().__init__(container, *args, **kwargs)
        self.placeholder = placeholder

        self.field_style = kwargs.pop("style", "TEntry")
        self.placeholder_style = kwargs.pop("placeholder_style", self.field_style)
        self["style"] = self.placeholder_style

        self.insert("0", self.placeholder)
        self.bind("<FocusIn>", self._clear_placeholder)
        self.bind("<FocusOut>", self._add_placeholder)

    def _clear_placeholder(self, e):
        if self["style"] == self.placeholder_style:
            self.delete("0", "end")
            self["style"] = self.field_style

    def _add_placeholder(self, e):
        if not self.get():
            self.insert("0", self.placeholder)
            self["style"] = self.placeholder_style

In [126]:
# Define globals and constants
HEIGHT = 400
WIDTH = 500
BG_COL = np.random.choice(['#9593cc','#fadc96','#cbfa96','#fa9896','#96f2fa','#fa96e4'])
current_artist = ''
all_data = pd.DataFrame(columns = ['artist','id','cover','name','release','track_count','track_info'])
current_data = pd.DataFrame(columns = ['artist','id','cover','name','release','track_count','track_info'])

# Define root properties
root = tk.Tk()
root.title('Comparify')
root.iconbitmap('spotify_icon_black.ico')
root.resizable(False, False)
canvas = tk.Canvas(root, height=HEIGHT, width=WIDTH)
canvas.pack()

# Frames
frame0 = tk.Frame(root,bg=BG_COL,bd=0) # frame containing search bar
frame0.place(relx=0.5, rely=0, relwidth=1,relheight=0.1,anchor='n')

frame1 = tk.Frame(root,bg=BG_COL,bd=0) # frame with main display
frame1.place(relx=0.5, rely = 0.1,relwidth=1,relheight=0.7,anchor='n')

frame2 = tk.Frame(root,bg=BG_COL,bd=2) # frame with generate button
frame2.place(relx=0.5, rely = 0.8,relwidth=1,relheight=0.2,anchor='n')

# Labels
s = 'Welcome to Artist Search\n\n'
s += 'use SEARCH (or enter) to search\n\n\
then press ADD (or Ctrl + /) if you would like \
to add that artist to compare\n\n\
pressing SAVE (or Ctrl + S) will save all\n\
current artist info in a csv\n\n\
pressing GENERATE (or Ctrl + G) will\n\
create graphs\n\n'
# using text over label for more customizability and scrolling
w = tk.Canvas(frame1, width=35,height=20)
w.pack(side=tk.LEFT,fill=tk.BOTH,padx=10,pady=10)

scrollbar = tk.Scrollbar(w,orient='vertical') # add scrollbar to display
display = tk.Text(w,font=('Calibri',10),bg='#ffffff',
                  width=35,height=20,
                  yscrollcommand = scrollbar.set)
display.config(state=tk.NORMAL)
display.insert(tk.END, s)
display.pack(side=tk.LEFT,fill=tk.BOTH)
display.config(state= tk.DISABLED)
scrollbar.pack(side = tk.RIGHT, fill = tk.Y)
scrollbar.config(command=display.yview) #for vertical scrollbar


t = 'Current Artists:\n'
c_artists = tk.Label(frame1,font=('Calibri',10),bg='#ffffff', text = t,justify='left',anchor='nw')
c_artists.pack(side=tk.LEFT,padx=10,pady=10)
c_artists.config(width=25,height=20)

# Entry fields
search_field = PlaceholderEntry(frame0,"Search Artist")
search_field.grid(row=0,column=0, padx=10, pady=10)
search_field.config(width = 32)

file_name = PlaceholderEntry(frame2,"File Name (optional)") # TO ADD
file_name.grid(row=3,column=0, padx=10, pady=10)
file_name.config(width = 32)

# Buttons
search_button = tk.Button(frame0, text='Search',command=lambda: search_API())
search_button.grid(row=0,column=1,padx=0, pady=10)
search_button.config(height = 1, width = 5)

add_button = tk.Button(frame0, text='Add',command=lambda: add_artist())
add_button.grid(row=0,column=2, padx=15, pady=10)
add_button.config(height = 1, width = 5)

## ADD CLEAR LIST BUTTON

generate_button = tk.Button(frame2, text='Generate',command=lambda: generate_plots())
generate_button.grid(row=3,column=1,padx=0, pady=10)
generate_button.config(height = 1, width = 12)

save_button = tk.Button(frame2, text='Save',command=lambda: save_data())
save_button.grid(row=3,column=2,padx=15, pady=10)
save_button.config(height = 1, width = 8)
root.bind('<Return>',search_API)
root.bind('<Control-slash>',add_artist)
root.bind('<Control-s>',save_data)
root.bind('<Control-g>',generate_plots)

root.mainloop()

You are generating standalone HTML/JS output, but trying to use real Python
callbacks (i.e. with on_change or on_event). This combination cannot work.

Only JavaScript callbacks may be used with standalone output. For more
information on JavaScript callbacks with Bokeh, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/interaction/callbacks.html

Alternatively, to use real Python callbacks, a Bokeh server application may
be used. For more information on building and running Bokeh applications, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/server.html

